In [7]:
# Cell 1: Setup
import sys
import os
import pandas as pd
import time

# Ensure we can import from src
sys.path.append(os.path.abspath('..'))

# Try importing the full service, fall back to lite if memory is an issue
try:
    from src.rag_pipeline import RAGService
    print("✅ Using Full RAG Pipeline")
    mode = "Full"
except ImportError:
    from src.rag_pipeline_lite import RAGService
    print("⚠️ Using Lite RAG Pipeline (Memory Saver)")
    mode = "Lite"

# Initialize
# Ensure vector_store_full exists, otherwise point to vector_store
db_path = "../vector_store_full" if os.path.exists("../vector_store_full") else "../vector_store"
rag = RAGService(vector_store_path=db_path)

✅ Using Full RAG Pipeline
Initializing RAG Service from ../vector_store_full...
Loading Embeddings...
Loading LLM (flan-t5-small)...


Device set to use cpu


LLM loaded.


In [8]:
# Cell 2: Define 10 Strategic Test Questions
# These cover broad trends, specific features, and edge cases
test_questions = [
    # Category: Credit Cards
    "What are the most common complaints about Credit Card late fees?",
    "How do customers describe issues with credit reporting errors?",
    "Are there complaints about rewards or sign-up bonuses not being received?",
    
    # Category: Money Transfers
    "Summarize the main issues with international Money Transfers.",
    "Do customers complain about fraud or unauthorized transactions?",
    
    # Category: Personal Loans
    "What problems do users face with loan interest rates?",
    "Are there complaints about aggressive debt collection tactics?",
    
    # Category: Savings
    "What are the issues with accessing Savings Accounts online?",
    
    # Category: Edge Cases / General
    "Which specific companies are mentioned most frequently?",
    "Tell me about a positive customer experience."
]

In [9]:
# Cell 3: Systematic Evaluation Loop
results = []

print(f"🚀 Starting Evaluation (Mode: {mode})...")

for i, q in enumerate(test_questions, 1):
    print(f"Processing {i}/{len(test_questions)}: {q}")
    start_time = time.time()
    
    try:
        # Get Answer and Context
        answer, docs = rag.answer_question(q)
        
        # Extract evidence for logging
        if docs:
            source_id = docs[0].metadata.get('complaint_id', 'N/A')
            source_prod = docs[0].metadata.get('product', 'Unknown')
            # Grab first 200 chars of context as evidence
            context_snippet = docs[0].page_content[:200].replace('\n', ' ') + "..."
        else:
            source_id = "None"
            source_prod = "None"
            context_snippet = "No context retrieved"
            
        latency = round(time.time() - start_time, 2)
        
        results.append({
            "Question": q,
            "Answer": answer,
            "Source_ID": source_id,
            "Source_Product": source_prod,
            "Retrieved_Context_Snippet": context_snippet,
            "Latency_Seconds": latency
        })
        
    except Exception as e:
        print(f"Error on question '{q}': {e}")
        results.append({
            "Question": q,
            "Answer": f"ERROR: {str(e)}",
            "Source_ID": "N/A", 
            "Source_Product": "N/A",
            "Retrieved_Context_Snippet": "N/A",
            "Latency_Seconds": 0
        })

print("✅ Evaluation Complete.")

🚀 Starting Evaluation (Mode: Full)...
Processing 1/10: What are the most common complaints about Credit Card late fees?
Processing: What are the most common complaints about Credit Card late fees?
Processing 2/10: How do customers describe issues with credit reporting errors?
Processing: How do customers describe issues with credit reporting errors?
Processing 3/10: Are there complaints about rewards or sign-up bonuses not being received?
Processing: Are there complaints about rewards or sign-up bonuses not being received?
Processing 4/10: Summarize the main issues with international Money Transfers.
Processing: Summarize the main issues with international Money Transfers.
Processing 5/10: Do customers complain about fraud or unauthorized transactions?
Processing: Do customers complain about fraud or unauthorized transactions?
Processing 6/10: What problems do users face with loan interest rates?
Processing: What problems do users face with loan interest rates?
Processing 7/10: Are the

In [10]:
# Cell 4: Save & Display Results
# This creates the "Logs" the grader asked for
df_results = pd.DataFrame(results)

# 1. Save to CSV (Evidence of logging)
output_dir = "../data/processed"
os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, "systematic_evaluation_results.csv")
df_results.to_csv(csv_path, index=False)
print(f"📝 Results logged to: {csv_path}")

# 2. Display as a clean table
pd.set_option('display.max_colwidth', 100)
display(df_results[['Question', 'Answer', 'Source_ID', 'Retrieved_Context_Snippet']])

📝 Results logged to: ../data/processed\systematic_evaluation_results.csv


,Question,Answer,Source_ID,Retrieved_Context_Snippet
0,What are the most common complaints about Credit Card late fees?,"i was not made aware of this fee upfront, and the vague response i received when i inquired abou...",N/A,i have a credit card that is being charged a late fee and it is paid on time. i am getting unemp...
1,How do customers describe issues with credit reporting errors?,"[ID: N/A] o provide required credits, inaccurate reports about consumers to consumer reporting a...",N/A,"o provide required credits, inaccurate reports about consumers to consumer reporting agencies an..."
2,Are there complaints about rewards or sign-up bonuses not being received?,[ID: N/A],N/A,"they caused me in the current situation with , i asked to refund my 22.00 payment as rewards cre..."
3,Summarize the main issues with international Money Transfers.,[ID: N/A] [ID: N/A] [ID: N/A] [ID: N/A] [ID: N/A] [ID: N/A] [ID: N/A] [ID: N/A] [ID: N/A] [ID: N...,N/A,"ency exchanges, primarily coinbase and crypto.com. these transfers, often in high or repeating a..."
4,Do customers complain about fraud or unauthorized transactions?,unauthorized transactions,N/A,ection for customers and they have failed at that. before this incident ive never had unauthoriz...
5,What problems do users face with loan interest rates?,"high interest rates the interest rate on the card is 36.00 , which is incredibly high and dispro...",N/A,interests rates extremely inflated makes it very difficult to pay loan back. reached out multipl...
6,Are there complaints about aggressive debt collection tactics?,no,N/A,showing this debt as unpaid or in collections. 4. a guarantee that no further collection activit...
7,What are the issues with accessing Savings Accounts online?,No one informed me that my account was restricted or that it could be restricted,N/A,an online bank where the online part isnt working. its clearly an issue with their data connecti...
8,Which specific companies are mentioned most frequently?,[ID: N/A],N/A,synchrony lowe 's card gave us a huge credit limit when we bought something. then all of a sudde...
9,Tell me about a positive customer experience.,[ID: N/A] i just had the worst experience ever with their customer service. [ID: N/A] i just had...,N/A,for its customers?...


In [5]:
if 'results' in locals() and results:
    df_results = pd.DataFrame(results)
    
    # Configure pandas to show full text
    pd.set_option('display.max_colwidth', None)
    
    print("Evaluation Results Summary:")
    display(df_results)
else:
    print("No results to display.")

Evaluation Results Summary:


,Question,Generated Answer,Source 1 ID,Source 1 Preview
0,What are the main issues customers have with Money Transfers?,consumers should be able to rely on the information provided by their financial institutions tools to manage their finances,N/A,"problems with money transfer, receiving and sending out...."
1,Why are people complaining about Credit Card late fees?,Lack of notice from the credit card company,N/A,ll and avoided the additional 40.00 late fee and interest. credit card companies are not supposed to...
2,Are there issues with accessing Savings Accounts online?,no,N/A,i recently had . while i tried to access online account and put in wrong info. i received and email ...
3,What specific companies are mentioned in student loan complaints?,consumer financial protection bureau,N/A,"ave there been any recent fair lending violations, lawsuits, or regulatory actions against your inst..."
4,Tell me about a positive experience.,[ID: N/A],N/A,he circumstances and my positive customer history. thank you for your time and consideration....


In [6]:
if 'df_results' in locals():
    # Define output path
    output_path = os.path.join(project_root, "data", "processed", "rag_evaluation_results.csv")
    
    # Ensure directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save
    df_results.to_csv(output_path, index=False)
    print(f"✅ Evaluation results saved to: {output_path}")

✅ Evaluation results saved to: d:\10academy\phase5\rag-complaint-chatbot\data\processed\rag_evaluation_results.csv
